# Thermostat demo

In [1]:
import warnings
# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
!pip3 install --upgrade pip
!pip3 install cmake
!pip3 install cython
!pip3 install numpy
!pip3 install torch
!pip3 install datasets
!pip3 install spacy
!pip3 install sentencepiece
!pip3 install transformers
!pip3 install overrides
!pip3 install jsonnet

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import sys
# Include root directory in module path
sys.path.append('src')

from pprint import pprint

import thermostat  # Accompanying library

# Load dataset

This will use the dataset script ("thermostat.py") in the "thermostat" directory.
In this example, we use the test set of the "imdb-bert-lig" configuration.
This refers to Layer Integrated Gradients (LIG) explanations of the predictions by a BERT model that has been fine-tuned on the IMDb (train) dataset and evaluated on the IMDb test dataset.
In other words, we load the 25k test examples from the IMDb test plus the BERT predictions and the feature attributions from a Layer Integrated Gradients explainer.

In [4]:
data = thermostat.load("imdb-bert-lig")
print(data)

Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/imdb-bert-lig/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e)


Loading Thermostat configuration: imdb-bert-lig
IMDb dataset, BERT model, Layer Integrated Gradients explanations
Explainer: LayerIntegratedGradients
Model: textattack/bert-base-uncased-imdb
Dataset: imdb



Now let's inspect a single instance of the loaded dataset.
Here, we will stick to the functionality that the datasets library already supplies us with.
For readability purposes, we will not print the whole content of that instance.
Instead, we're showing only the first few entries of the attributions and the input_ids, respectively.

In [5]:
example = data[0]

print(f'Index: {example.index}')
print(f'Attributions (first 4): {example.attributions[:4]}')
print(f'True label: {example.true_label}')
print(f'Predicted label: {example.predicted_label}')

Tokenizing imdb-bert-lig instances (Tokenizer: textattack/bert-base-uncased-imdb): 100%|██████████| 25000/25000 [00:13<00:00, 1920.34it/s]

Index: 0
Attributions (first 4): [0.0, -0.18146862089633942, 0.13141055405139923, 0.2554049491882324]
True label: {'index': 1, 'name': 'pos'}
Predicted label: {'index': 1, 'name': 'pos'}


In [6]:
print(f'Accuracy of {data.model_name} :')
sum([b_i.true_label == b_i.predicted_label for b_i in data])/len(data)

Accuracy of textattack/bert-base-uncased-imdb :


0.93028

In [7]:
print(f'Texts:\n{example.texts}')

Texts:
text: i went and saw this movie last night after being coa ##xed to by a few friends of mine . i ' ll admit that i was reluctant to see it because from what i knew of ashton ku ##tch ##er he was only able to do comedy . i was wrong . ku ##tch ##er played the character of jake fischer very well , and kevin cost ##ner played ben randall with such professional ##ism . the sign of a good movie is that it can toy with our emotions . this one did exactly that . the entire theater ( which was sold out ) was overcome by laughter during the first half of the movie , and were moved to tears during the second half . while exiting the theater i not only saw many women in tears , but many full grown men as well , trying desperately not to let anyone see them crying . this movie was great , and i suggest that you go see it before you judge .


# Visualize data
Can we make this a bit more readable?  
Of course!  


In [8]:
example.render(jupyter=True)

Heatmap of text field text


# Inspecting misclassified instances

In [9]:
misclassified = [instance for instance in data if instance.true_label != instance.predicted_label]
print(misclassified[0].true_label, misclassified[0].predicted_label)
misclassified[0].render(jupyter=True)
misclassified[0].heatmap

{'index': 1, 'name': 'pos'} {'index': 0, 'name': 'neg'}
Heatmap of text field text


{'text': Token: ", Color: (255,252,252), Score: 0.011
 Token: night, Color: (229,229,255), Score: -0.098
 Token: of, Color: (176,176,255), Score: -0.308
 Token: the, Color: (222,222,255), Score: -0.129
 Token: hunted, Color: (252,252,255), Score: -0.008
 Token: ", Color: (179,179,255), Score: -0.296
 Token: stars, Color: (211,211,255), Score: -0.17
 Token: french, Color: (255,231,231), Score: 0.094
 Token: porn, Color: (255,217,217), Score: 0.146
 Token: star, Color: (244,244,255), Score: -0.043
 Token: brigitte, Color: (246,246,255), Score: -0.033
 Token: lahaie, Color: (243,243,255), Score: -0.046
 Token: ., Color: (240,240,255), Score: -0.058
 Token: in, Color: (255,195,195), Score: 0.234
 Token: fact, Color: (133,133,255), Score: -0.476
 Token: ,, Color: (255,221,221), Score: 0.131
 Token: many, Color: (224,224,255), Score: -0.118
 Token: of, Color: (255,254,254), Score: 0.002
 Token: the, Color: (243,243,255), Score: -0.047
 Token: cast, Color: (245,245,255), Score: -0.038
 Token:

# Aggregate data
Let us first compare the heatmaps of two different models on the same dataset+explainer configuration, MNLI + Occlusion.

In [9]:
bert = thermostat.load("multi_nli-bert-occ")
xlnet = thermostat.load("multi_nli-xlnet-occ")

Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-bert-occ/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e)
Loading cached processed dataset at /home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-bert-occ/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e/cache-f82aeb187fbdd1a1.arrow
Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-xlnet-occ/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e)
Loading cached processed dataset at /home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-xlnet-occ/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e/cache-f50d37ba0d4326b9.arrow


Loading Thermostat configuration: multi_nli-bert-occ
Loading Thermostat configuration: multi_nli-xlnet-occ


In [10]:
albert = thermostat.load("multi_nli-albert-occ")
electra = thermostat.load("multi_nli-electra-occ")
roberta = thermostat.load("multi_nli-roberta-occ")

Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-albert-occ/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e)


Loading Thermostat configuration: multi_nli-albert-occ


Loading cached processed dataset at /home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-albert-occ/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e/cache-f034a2f190cb8b1a.arrow
Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-electra-occ/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e)


Loading Thermostat configuration: multi_nli-electra-occ


Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-roberta-occ/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e)
Loading cached processed dataset at /home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-roberta-occ/1.0.0/81f1df4d48393ad0ff6fd17ad1ce534712170692290fccc657c931903f828d5e/cache-f1032f37283893cb.arrow


Loading Thermostat configuration: multi_nli-roberta-occ


In [11]:
albert[1].render(jupyter=True)
albert[1].true_label, albert[1].predicted_label

Tokenizing instances for multi_nli-albert-occ (Tokenizer: prajjwal1/albert-base-v2-mnli): 100%|██████████| 9815/9815 [00:07<00:00, 1245.69it/s]


({'index': 0, 'name': 'contradiction'}, {'index': 0, 'name': 'contradiction'})

In [12]:
bert[1].render(jupyter=True)
bert[1].true_label, bert[1].predicted_label

Tokenizing instances for multi_nli-bert-occ (Tokenizer: textattack/bert-base-uncased-MNLI): 100%|██████████| 9815/9815 [00:06<00:00, 1424.27it/s]


({'index': 0, 'name': 'contradiction'}, {'index': 0, 'name': 'contradiction'})

In [13]:
electra[1].render(jupyter=True)
electra[1].true_label, electra[1].predicted_label

Tokenizing instances for multi_nli-electra-occ (Tokenizer: howey/electra-base-mnli): 100%|██████████| 9815/9815 [00:07<00:00, 1286.73it/s]


({'index': 2, 'name': 'contradiction'}, {'index': 2, 'name': 'contradiction'})

In [14]:
roberta[1].render(jupyter=True)
roberta[1].true_label, roberta[1].predicted_label

Tokenizing instances for multi_nli-roberta-occ (Tokenizer: textattack/roberta-base-MNLI): 100%|██████████| 9815/9815 [00:07<00:00, 1324.82it/s]


({'index': 0, 'name': 'contradiction'}, {'index': 0, 'name': 'contradiction'})

In [15]:
xlnet[1].render(jupyter=True)
xlnet[1].true_label, xlnet[1].predicted_label

Tokenizing instances for multi_nli-xlnet-occ (Tokenizer: textattack/xlnet-base-cased-MNLI): 100%|██████████| 9815/9815 [00:07<00:00, 1288.03it/s]


({'index': 0, 'name': 'contradiction'}, {'index': 0, 'name': 'contradiction'})

### Rank correlation
We can also investigate how well two explainers on the same dataset+model combination correlate regarding their attribution scores.